## Introduction

This notebook demonstrates how to implement parallel function calling using Anthropic Claude Sonnet on Amazon Bedrock using the Boto3 API

It builds upon the multi agent implementation example from [here](bedrock-converse-multi-agent.ipynb)

In [149]:
!pip install python-dotenv --quiet

In [150]:
import boto3
from dotenv import load_dotenv
import concurrent.futures
import requests
import os

# Load .env file
load_dotenv()

True

## Define tools for the agent

In [151]:
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

def get_current_weather(input) -> int:
    """Get the current temperature from a city, in Fahrenheit"""
    
    city = input["city"]
    country = input ["country"]
    response = requests.get(
        f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={OPENWEATHERMAP_API_KEY}"
    )
    data = response.json()
    temp_kelvin = data["main"]["temp"]
    temp_fahrenheit = (temp_kelvin - 273.15) * 9 / 5 + 32
    return int(temp_fahrenheit)


def get_difference(input) -> int:
    """Get the difference between two numbers"""
    
    minuend = input["minuend"]
    subtrahend = input["subtrahend"]
    return minuend - subtrahend


tools = [
    {
        "toolSpec": {
            "name": "get_current_weather",
            "description": "Get the current temperature from a city, in Fahrenheit",
            # "description": "This tool returns the temperature of city in fahrenheit",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            # "description": "The city"
                            "description": "City"
                        },
                        "country": {
                            "type": "string",
                            # "description": "The country code"
                            "description": "Country Code"
                        }
                    },
                    "required": ["city", "country"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_difference",
            "description": "Get the difference between two numbers",
            # "description": "This tool substract and returns the difference between two numbers",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "minuend": {
                            "type": "integer",
                            "description": "The number from which another number is to be subtracted"
                        },
                        "subtrahend": {
                            "type": "integer",
                            "description": "The number to be subtracted"
                        }
                    },
                    "required": ["minuend", "subtrahend"]
                }
            }
        }
    }
]

## Define helper function to call Bedrock

In [152]:
def call_bedrock(system_message, message_list, tool_list):
    session = boto3.Session()

    bedrock = session.client(service_name='bedrock-runtime')
    
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=message_list,
        system=system_message,
        inferenceConfig={
            # "maxTokens": 2000,
            "temperature": 0
        },
        toolConfig={ "tools": tool_list }
    )
    
    return response

## Define helper function to invoke a given tool

In [153]:
def get_tool_result(tool_use_block):
    
    tool_name = tool_use_block['name']
    tool_input = tool_use_block['input']
    
    print(f"Using tool `{tool_name}` for query `{tool_input}`")
    
    func = globals()[tool_name]
    try:
        tool_result_value = func(tool_input)
        if tool_result_value is not None:
            return {
                "toolResult": {
                    "toolUseId": tool_use_block['toolUseId'],
                    "content": [
                        { "json": { "text": tool_result_value } }
                    ]
                }
            }
    except Exception as e:
        return { 
            "toolResult": {
                "toolUseId": tool_use_block['toolUseId'],
                "content": [  { "text": repr(e) } ],
                "status": "error"
            }
        }

    

## Define function to handle the raw responses from Bedrock

In [154]:
def handle_model_response(response):
    
    response_content_blocks = response['content']
    follow_up_content_blocks = []
    tool_use_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool_use_blocks.append(content_block['toolUse'])
    
    if len(tool_use_blocks) > 0:
            if len(tool_use_blocks) == 1:
                
                # If there's only one tool use, process it without using parallel execution
                tool_result_value = get_tool_result(tool_use_blocks[0])
                follow_up_content_blocks.append(tool_result_value)
            
            else:
                # If there are multiple elements, use parallel execution
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    # Submit tasks for each item in the input list
                    future_results = [executor.submit(get_tool_result, tool_use_block) for tool_use_block in tool_use_blocks]

                    # Collect results as they complete
                    for future in concurrent.futures.as_completed(future_results):
                        result = future.result()
                        follow_up_content_blocks.append(result)
         
    if len(follow_up_content_blocks) > 0:
        
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None

## Define function that implements the Agent Loop till final response is received

In [155]:
def run_agent(system_prompt, input_prompt, tool_list):
    # MAX_LOOPS = 6
    # loop_count = 0
    continue_loop = True
    output = ""
    
    system_message = [
        {
            "text": system_prompt
        }
    ]
    
    message_list = [
        {
            "role": "user",
            "content": [ { "text": input_prompt } ]
        }
    ]
    
    while continue_loop:
        response = call_bedrock(system_message, message_list, tool_list)

        response_message = response['output']['message']
        message_list.append(response_message)

        # loop_count = loop_count + 1

        # if loop_count >= MAX_LOOPS:
        #     print(f"Hit loop limit: {loop_count}")
        #     break

        follow_up_message = handle_model_response(response_message)

        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False
            if response['stopReason'] == "end_turn":
                agent_output = response_message['content'][0]['text']
        else:
            message_list.append(follow_up_message)
            
    return message_list, agent_output

## Define agent executor

In [156]:
# Agent Executor

def agent_executor(agent_input_prompt):
    
    agent_system_prompt = "You are a helpful assistant that answer questions about weather"

    agent_workflow, agent_output = run_agent(agent_system_prompt, agent_input_prompt, tools)
    
    # returning just the output for now
    # can choose to return the workflow `agent_workflow` as well which includes details of the different tools called when this agent is called
    
    return agent_output

## Run Agent

In [157]:
agent_executor("Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?")

Using tool `get_current_weather` for query `{'city': 'Austin', 'country': 'US'}`
Using tool `get_current_weather` for query `{'city': 'Tokyo', 'country': 'JP'}`
Using tool `get_current_weather` for query `{'city': 'Seattle', 'country': 'US'}`
Using tool `get_difference` for query `{'minuend': 89, 'subtrahend': 80}`
Using tool `get_difference` for query `{'minuend': 89, 'subtrahend': 63}`


'Austin at 89°F is the warmest of the three cities. It is 9°F warmer than Tokyo, and 26°F warmer than Seattle.'